# Clustering

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

# KMEANS
from sklearn.cluster import KMeans
from yellowbrick.cluster.elbow import KElbowVisualizer 
from yellowbrick.cluster import silhouette_visualizer 

# DBSCAN
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import pdist, squareform
from sklearn.neighbors import NearestNeighbors

# HIERARCHICAL
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering

# EMA - XMEANS - FCM
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from pyclustering.cluster.ema import ema, ema_initializer, ema_init_type
from pyclustering.cluster.xmeans import xmeans
from pyclustering.cluster.fcm import fcm

# Visualization
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import seaborn as sns
pd.options.plotting.backend = "plotly"
pio.templates.default = "seaborn"

pd.set_option('display.max_columns', None)

In [ ]:
# Utiliy functions
interesting_features = ['mean_rank_points', 'lrpOnAvgrp', 'age', 'total_matches_played']

def show_pca_visualization(cluster_type: str):
  df = df_players[df_players.select_dtypes(include = np.number).columns.tolist()].drop(columns = ['ht', 'mean_minutes', 'max_minutes', 'rel_ace', 'rel_df', 'rel_1stIn', 'rel_1stWon', 'rel_2ndWon', '1WonOn1In', '1WonOnTotWon', 'rel_ptsWon', 'rel_bpFaced', 'rel_bpSaved', 'rel_gmsWon'])
  df = pd.DataFrame(MinMaxScaler().fit_transform(df), columns=df.columns)
  components_df = pd.DataFrame(PCA(n_components=2).fit_transform(df))
  px.scatter(x=components_df[0], y=components_df[1], color=df_players[cluster_type]).show()

def show_interpretation_table(cluster_type: str):
  return df_players.groupby(cluster_type).agg({cluster_type:"count", "mean_rank_points": "mean", "lrpOnAvgrp": "mean", "age": "mean", "matches_won_ratio": "mean", "total_matches_played": "mean"}).sort_values(by="mean_rank_points", ascending=False).round(2).rename(columns={cluster_type: "cluster size"})

## Preparation

In [ ]:
df_players = pd.read_csv("./datasets/players.csv", index_col=0)
feautures = ['max_tourney_revenue', 'mean_rank_points', 'lrpOnMxrp', 'matches_won_ratio']

In [ ]:
for feature in feautures:
    df_players[feature].hist().show()

### Normalization

In [ ]:
df_data = df_players[feautures].reset_index(drop=True)
df_data = df_data.round(3)

# Transformations
#df_data['mean_rank_points'] = np.log(df_data['mean_rank_points'])

# Plot
df_data['mean_rank_points'].hist().show()
df_data = pd.DataFrame(MinMaxScaler().fit_transform(df_data), columns=df_data.columns)
df_data.boxplot(column=feautures).show()

## K-means

### Find Optimal K

In [ ]:
model = KMeans(n_init=10, max_iter=100, init="k-means++")
sse_visualizer = KElbowVisualizer(model, k=(2,8), timings=False)
sse_visualizer.fit(df_data)
sse_visualizer.show()

sil_visualizer = KElbowVisualizer(model, k=(2,8), timings=False, metric="silhouette")
sil_visualizer.fit(df_data)
sil_visualizer.show()

Picking optimal K

The optimal `k` is 4

In [ ]:
optimal_k = sse_visualizer.elbow_value_
kmeans = KMeans(n_clusters=optimal_k, n_init=10, max_iter=100, init="k-means++")
kmeans.fit(df_data)

df_players["cluster_kmeans"] = kmeans.labels_.astype(str)

x = silhouette_visualizer(KMeans(optimal_k, random_state=42), df_data)
print("The silhoutte score is: " + str(x.silhouette_score_))

### Result analysis

In [ ]:
show_interpretation_table("cluster_kmeans")

Plot of the k-means centers

In [ ]:
plt.figure(figsize=(15, 4))
for i in range(0, len(kmeans.cluster_centers_)):
    plt.plot(kmeans.cluster_centers_[i], marker='o', label='Cluster %s' % i)
plt.xticks(range(0, len(df_data.columns)), df_data.columns, fontsize=15)
plt.legend(fontsize=10)
plt.show()

#### Box plots

In [ ]:
px.box(df_players, y="total_matches_played", color="cluster_kmeans").show()
px.box(df_players, y="age", color="cluster_kmeans").show()
px.box(df_players, y="max_performance_index", color="cluster_kmeans").show()
#px.histogram(df_players, x="cluster_kmeans", y="total_matches_played", color="cluster_kmeans").show()

#### PCA visualization

In [ ]:
show_pca_visualization(cluster_type="cluster_kmeans")

#### Scatter matrix of selected features

In [ ]:
px.scatter_matrix(df_players,
    dimensions=feautures,
    color="cluster_kmeans")

#### Scatter matrix of interesting features

In [ ]:
px.scatter_matrix(df_players,
    dimensions=interesting_features,
    color="cluster_kmeans")

#### Histograms of interesting features by gender
The only important difference between male and female players that can be seen is that female players tend to be more than the counterpart, nevertheless no discrimination is made

In [ ]:
# to plot the histograms mean reank points is visualize as log(mean_rank_points) to better appreciate the results
df_players_to_visualize = df_players
interesting_features_to_visualize = ['log_mean_rank_points', 'lrpOnAvgrp', 'age', 'total_matches_played']
df_players_to_visualize['log_mean_rank_points'] = np.log(df_players_to_visualize['mean_rank_points'])

for feature in interesting_features_to_visualize:
  px.histogram(df_players_to_visualize, x=feature, facet_col="cluster_kmeans", color=df_players.gender).show()

Clustering does not clearly distinguish between classes of players, however it is possible to find fairly defined patterns by observing the following histograms (6, 7, 8, 9) and the plot related to the centroids and the plot regarding the centroids (10).

- Cluster 0 represents the young promises: those with low mean rank points, an average low age and on average the ones with the strongest trends of growth (looking at the lrpOnAvgrp).
- Cluster 1 represent the strongest players: with experience and a generally higher age. They have high mean rank points and perform the best in term of matches won ratio.
- Cluster 2 represent good players with a decreasing trend.
- Cluster 3: represents the bad players: with low mean rank points and a decreasing trend.

#### Practical interpretation

Looking at some example we can get a more practical idea of the clusters, in this case distinguishing cluster 1 from 2.

Among all the possible examples we show the most famous / strong players so that you can better understand the differences

In [ ]:
df_players[df_players['cluster_kmeans'] == '1'].sort_values(by='mean_rank_points', ascending=False).loc[:, ['name', 'mean_rank_points', 'age', 'lrpOnAvgrp']].head()

In [ ]:
df_players[df_players['cluster_kmeans'] == '2'].sort_values(by='mean_rank_points', ascending=False).loc[:, ['name', 'mean_rank_points', 'age', 'lrpOnAvgrp']].head()

## Density-based

In [ ]:
# pair-wise distance and then compute distance matrix
dist = pdist(X=df_data, metric='euclidean')
dist = squareform(dist)


kth_distances = {k:[] for k in range(2, 10 +1, 4)}
for kth_distance in range(20, 60, 10):
    kth_distances[kth_distance] = []

for d in dist:
    indexes_to_sort_d = np.argsort(d)
    for k in kth_distances:
        kth_distances[k].append(d[indexes_to_sort_d[k]])

fig = go.Figure()
for k in kth_distances.keys():
    fig.add_trace(go.Scatter(x = np.array(range(0, len(kth_distances[k]))), y = sorted(kth_distances[k]), mode = 'lines' , name = str(k)))
fig.show()

### Find optimal hyper-parameters

In [ ]:
def get_metrics(eps, min_samples, dataset, iter_):
    # Fit the model
    dbscan_model_ = DBSCAN( eps = eps, min_samples = min_samples)
    dbscan_model_.fit(dataset)
    
    # Mean noise point distance metric
    noise_indices = dbscan_model_.labels_ == -1
    
    if True in noise_indices:
        neighboors = NearestNeighbors(n_neighbors = 6).fit(dataset)
        distances, indices = neighboors.kneighbors(dataset)
        noise_distances = distances[noise_indices, 1:]
        noise_mean_distance = round(noise_distances.mean(), 3)
    else:
        noise_mean_distance = None
        
    # Number of clusters metric
    number_of_clusters = len(set(dbscan_model_.labels_[dbscan_model_.labels_ >= 0]))

    # Silhouette score
    if (number_of_clusters <= 1):
        sil = 0
    else:
        sil = silhouette_score(dataset, dbscan_model_.labels_)

    return(noise_mean_distance, number_of_clusters, sil)

eps_to_test = [0.01]
for eps in np.arange(0.05, 0.30, 0.05):
    eps_to_test.append(round(eps,3))

min_samples_to_test = [2, 4, 6, 8, 10]
for min_samples in range(20, 60, 10):
    min_samples_to_test.append(min_samples)

# Dataframe per la metrica sulla distanza media dei noise points dai K punti più vicini
results_noise = pd.DataFrame( 
    data = np.zeros((len(eps_to_test),len(min_samples_to_test))), # Empty dataframe
    columns = min_samples_to_test, 
    index = eps_to_test
)

# Dataframe per la metrica sul numero di cluster
results_clusters = pd.DataFrame( 
    data = np.zeros((len(eps_to_test),len(min_samples_to_test))), # Empty dataframe
    columns = min_samples_to_test, 
    index = eps_to_test
)

# Dataframe to store the silhouette score for each combo in the grid search
results_silhouette = pd.DataFrame(
    data = np.zeros((len(eps_to_test), len(min_samples_to_test))),
    columns = min_samples_to_test,
    index = eps_to_test
)

iter_ = 0
for eps in eps_to_test:
    for min_samples in min_samples_to_test:
        iter_ += 1
        # Calcolo le metriche
        noise_metric, cluster_metric, silhouette_metric = get_metrics(eps, min_samples, df_data, iter_)

        results_noise.loc[eps, min_samples] = noise_metric
        results_clusters.loc[eps, min_samples] = cluster_metric
        results_silhouette.loc[eps, min_samples] = silhouette_metric

In [ ]:
#sm = (results_clusters >= 2) & (results_clusters <= 3)
# sm = (results_clusters == 3)
sm = (results_clusters >=0 )
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(50,10) )
sns.set(font_scale=2.5)
sns.heatmap(results_noise[sm], annot = True, ax = ax1, cbar = False, fmt='.2f').set_title("METRIC: Mean Noise Points Distance")
sns.heatmap(results_clusters[sm], annot = True, ax = ax2, cbar = False).set_title("METRIC: Number of clusters")
sns.heatmap(results_silhouette[sm], annot = True, ax = ax3, cbar = False).set_title("METRIC: Silhouette score")
ax1.set_xlabel("N"); ax2.set_xlabel("N") ; ax3.set_xlabel("N")
ax1.set_ylabel("EPSILON"); ax2.set_ylabel("EPSILON") ; ax3.set_ylabel("EPSILON")
plt.tight_layout(); plt.show()
sns.set(font_scale=1)

In [ ]:
# dbscan = DBSCAN(eps=0.9, min_samples=3).fit(df_data)
# dbscan = DBSCAN(eps=0.4, min_samples=29).fit(df_data)
#dbscan = DBSCAN(eps=0.2, min_samples=5).fit(df_data)
dbscan = DBSCAN(eps=0.2, min_samples=6).fit(df_data)

results = np.unique(dbscan.labels_, return_counts=True)
print(f"Clusters labels: {results[0]}\nElements per cluster: {results[1]}")
df_players["cluster_dbscan"] = dbscan.labels_.astype(str)

### Result analysis

In [ ]:
show_interpretation_table("cluster_dbscan")

Overview of noise points:

In [ ]:
df_players[df_players['cluster_dbscan'] == '-1'][interesting_features]

#### PCA visualization

In [ ]:
show_pca_visualization(cluster_type="cluster_dbscan")

#### Scatter matrix of selected features

In [ ]:
px.scatter_matrix(df_players,
    dimensions=feautures,
    color="cluster_dbscan")

#### Scatter matrix of interesting features

In [ ]:
px.scatter_matrix(df_players,
    dimensions=interesting_features,
    color="cluster_dbscan")

In the following table we can appreciate how DBSCAN discriminated noise points from the the other samples:

In [ ]:
px.scatter(df_players, x='mean_rank_points', y='total_matches_played', color = 'cluster_dbscan')

#### Interpretation
The clusters result fairly balanced in the number of elements and they represented either good or bad players.
The noise labelled by DBSCAN was none other than very bad o very good player.

## Hierarchical

### Utility functions

In [ ]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram
    # create the counts of samples under each node
    
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [ ]:
# print some info regarding the clustering and add 'cluster_hierarchical' attribute to dataframe
def info(model, df_players, df_data):
    results = np.unique(model.labels_, return_counts=True)
    print(f"Clusters labels: {results[0]}\nElements per cluster: {results[1]}")

    df_players["cluster_hierarchical"] = model.labels_.astype(str)
    #df_players = df_players[df_players.select_dtypes(include = np.number).columns.tolist()].drop(columns = ['ht', 'mean_minutes', 'max_minutes', 'rel_ace', 'rel_df', 'rel_1stIn', 'rel_1stWon', 'rel_2ndWon', '1WonOn1In', '1WonOnTotWon', 'rel_ptsWon', 'rel_bpFaced', 'rel_bpSaved', 'rel_gmsWon'])
    print("Silhouette score: " + str(silhouette_score(df_data, model.labels_.astype(str))))

### Ward

In [ ]:
threshold_value = 25

# plot the full dendogram
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage='ward')
model = model.fit(df_data)
plot_dendrogram(model, truncate_mode="level", color_threshold=threshold_value, p=4)
if threshold_value != None:
    plt.axhline(y=threshold_value, color="black")
plt.show()

In [ ]:
# run again agglomerating up to 2 clusters
model = AgglomerativeClustering(n_clusters=2, linkage='ward')
model = model.fit(df_data)

#### Results analysis

In [ ]:
info(model, df_players, df_data)
show_interpretation_table("cluster_hierarchical")

In [ ]:
px.scatter_matrix(df_players,
    dimensions=interesting_features,
    color="cluster_dbscan")

In [ ]:
show_pca_visualization(cluster_type="cluster_hierarchical")

### Complete

In [ ]:
threshold_value = 1.6

# plot the full dendogram
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage='complete')
model = model.fit(df_data)
plot_dendrogram(model, truncate_mode="level", color_threshold=threshold_value, p=4)
if threshold_value != None:
    plt.axhline(y=threshold_value, color="black")
plt.show()

In [ ]:
# run again agglomerating up to 3 clusters
model = AgglomerativeClustering(n_clusters=4, linkage='complete')
model = model.fit(df_data)

#### Results analysis

In [ ]:
info(model, df_players, df_data)
show_interpretation_table("cluster_hierarchical")

### Single

In [ ]:
threshold_value = 0.25

# plot the full dendogram
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage='single')
model = model.fit(df_data)
plot_dendrogram(model, truncate_mode="level", color_threshold=threshold_value, p=4)
if threshold_value != None:
    plt.axhline(y=threshold_value, color="black")
plt.show()

In [ ]:
# run again agglomerating up to 4 clusters
model = AgglomerativeClustering(n_clusters=4, linkage='ward')
model = model.fit(df_data)

#### Results analysis

In [ ]:
info(model, df_players, df_data)
show_interpretation_table("cluster_hierarchical")

### Average

In [ ]:
threshold_value = 0.7

# plot the full dendogram
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage='average')
model = model.fit(df_data)
plot_dendrogram(model, truncate_mode="level", color_threshold=threshold_value, p=4)
if threshold_value != None:
    plt.axhline(y=threshold_value, color="black")
plt.show()

In [ ]:
# run again agglomerating up to 3 clusters
model = AgglomerativeClustering(n_clusters=3, linkage='ward')
model = model.fit(df_data)

#### Results analysis

In [ ]:
info(model, df_players, df_data)
show_interpretation_table("cluster_hierarchical")

## EMA

In [ ]:
df = df_data.values.tolist()
amount_clusters=2
initial_means, initial_covariance = ema_initializer(df, amount_clusters).initialize(ema_init_type.KMEANS_INITIALIZATION)
ema_instance = ema(df, 2, initial_means, initial_covariance, tolerance=100)
ema_instance.process()

# Get clustering results.
clusters = ema_instance.get_clusters()
covariances = ema_instance.get_covariances()
means = ema_instance.get_centers()

for i, cluster in enumerate(clusters):
    print(f"Cluster {i}: {len(cluster)}")

for i, cluster in zip(range(len(clusters)), clusters):
    df_players.loc[df_players.index[cluster], 'cluster_gm'] = str(i)

In [ ]:
show_pca_visualization("cluster_gm")
show_interpretation_table("cluster_gm")

## X-MEAN

In [ ]:
# Create initial centers for xmeans algorithm
amount_initial_centers = 2
maximum_clusters = 40
initial_centers = kmeans_plusplus_initializer(df_data, amount_initial_centers, random_state=42).initialize()
 
# Create instance of X-Means algorithm with Bayesian Information Criterion (BIC) splitting criterion.
xmeans_instance = xmeans(df_data, initial_centers, maximum_clusters, tolerance=0.5, random_state=47)
xmeans_instance.process()
 
# Extract clustering results: clusters and their centers
clusters = xmeans_instance.get_clusters()
centers = xmeans_instance.get_centers()
 
# Print total sum of metric errors
print("Total WCE:", xmeans_instance.get_total_wce())

# Add results to dataset
xmeans_clusters = [] 
for i in range(0, len(clusters)):
    for j in clusters[i]:
        xmeans_clusters.append((j, i))

# sort according to first element of tuple
xmeans_clusters.sort(key=lambda tup: tup[0])
# keep only second element of tuple
xmeans_clusters = np.array([tup[1] for tup in xmeans_clusters])

df_players["cluster_xmeans"] = xmeans_clusters.astype(str)
print("clusters found: " + str(len(clusters)) + " on a maximum of " + str(maximum_clusters))

### Results analysis

In [ ]:
show_pca_visualization("cluster_xmeans")
show_interpretation_table("cluster_xmeans")

### Result analysis

In [ ]:
plt.figure(figsize=(15, 4))
for i in range(0, len(centers)):
    plt.plot(centers[i], marker='o', label='Cluster %s' % i)
plt.xticks(range(0, len(df_data.columns)), df_data.columns, fontsize=15)
plt.legend(fontsize=10)
plt.show()

In [ ]:
px.scatter_matrix(df_players,
    dimensions=feautures,
    color="cluster_kmeans")

## Fuzzy C-means

In [ ]:
# Prepare initial centers - amount of initial centers defines amount of clusters from which X-Means will
# start analysis.
amount_initial_centers = 4
initial_centers = kmeans_plusplus_initializer(df_data, amount_initial_centers, random_state=42).initialize()

# Create instance of Fuzzy C-Means algorithm, run it and get results
fcm_instance = fcm(df_data.to_numpy(), initial_centers, m=1.5)
fcm_instance.process()
clusters = fcm_instance.get_clusters()
centers = fcm_instance.get_centers()

# Add cluster label to original dataframe
fcm_clusters = [] 
for i in range(0, len(clusters)):
    for j in clusters[i]:
        fcm_clusters.append((j, i))

# sort according to first element of tuple
fcm_clusters.sort(key=lambda tup: tup[0])
# keep only second element of tuple
fcm_clusters = np.array([tup[1] for tup in fcm_clusters])

df_players["cluster_fcm"] = fcm_clusters.astype(str)
show_interpretation_table("cluster_fcm")

The fuzzy c-means returns the same results from the k-means and that was expeceted given the fact that the real difference is not in the `fcm_instance.get_clusters()` method but in `fcm_instance.get_membership()` that takes in account the fuzzyness